In [1]:
import pandas as pd
import numpy as np
import sys
import plotly.express as px
sys.path.append('../utils/')
from clean import CleanCode
clean = CleanCode()

In [2]:
def clean_code(df):
    df['Country'] = df['Country Code'].apply(clean.country_name)
    df['price_type'] = df['Price range'].apply(clean.create_price_type)
    df['Colors'] = df['Rating color'].apply(clean.color_name)
    df = clean.rename_columns(df)
    df = df.dropna()
    df["cuisines"] = df.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])

    return df

In [3]:
df = pd.read_csv('../data/zomato.csv')

### Tratando so dados

In [4]:
df.shape

(7527, 21)

In [5]:
df = clean_code(df)
df = df.drop_duplicates()

In [6]:
df.shape

(6929, 24)

In [7]:
df.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'country', 'price_type', 'colors'],
      dtype='object')

In [8]:
df.head()

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes,country,price_type,colors
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,Philippines,expensive,darkgreen
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,0,0,4,4.7,3F7E00,Excellent,469,Philippines,gourmet,darkgreen
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,...,0,0,3,4.4,5BA829,Very Good,867,Philippines,expensive,green
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,0,0,3,4.4,5BA829,Very Good,858,Philippines,expensive,green
5,6304833,Manam,162,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.020380,14.552351,Filipino,...,0,0,3,4.7,3F7E00,Excellent,930,Philippines,expensive,darkgreen


### Respondendo as Perguntas

#### Geral

1. Quantos restaurantes únicos estão registrados?

In [9]:
unicos = df['restaurant_id'].nunique()
unicos

6929

2. Quantos países únicos estão registrados?

In [10]:
unicos = len(df.loc[:, 'country'].unique())
unicos

15

3. Quantas cidades únicas estão registradas?

In [11]:
unicos = len(df.loc[:, 'city'].unique())
unicos

125

4. Qual o total de avaliações feitas?

In [12]:
total = df['votes'].sum()
total

4194533

5. Qual o total de tipos de culinária registrados?

In [13]:
unicos = len(df.loc[:, 'cuisines'].unique())
unicos

165

#### Pais

1. Qual o nome do país que possui mais cidades registradas?

In [17]:
cols = ['city','country']

pais = df.loc[:,cols].groupby('country').nunique().sort_values('city',ascending=False).reset_index()
pais.head(1)

,country,city
0,India,49


In [18]:
#pais = df.groupby('country')['city'].nunique().reset_index()
#pais = pais.sort_values('city', ascending=False).reset_index(drop=True)
#pais


2. Qual o nome do país que possui mais restaurantes registrados?

In [19]:
pais = df.groupby('country')['restaurant_id'].count().sort_values(ascending=False).reset_index()
pais.head(1)

,country,restaurant_id
0,India,3111


3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4

In [20]:
cols = ['restaurant_name', 'country', 'price_range']

pais = df.loc[df['price_range'] == 4, ['restaurant_name', 'country']].groupby('country').nunique().sort_values('restaurant_name', ascending=False).reset_index()
pais.head(1)

,country,restaurant_name
0,United States of America,390


4. Qual o nome do país que possui a maior quantidade de tipos de culinária
distintos?

In [21]:
nome_pais = df.groupby('country')['cuisines'].nunique().sort_values(ascending=False).reset_index()
nome_pais.head(1)

,country,cuisines
0,India,77


In [22]:
#pais = df.groupby('country')['cuisines'].nunique().reset_index()
#pais = pais.sort_values('cuisines', ascending=False).reset_index(drop=True)
#pais.head(1)

5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [23]:
pais = df.groupby('country')['votes'].sum().reset_index()
pais = pais.sort_values('votes', ascending=False).reset_index(drop=True)
pais.head(1)

,country,votes
0,India,2800164


6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem
entrega?

In [24]:
pais = df[df['has_online_delivery'] == 1].groupby('country')['restaurant_id'].count().reset_index()
pais = pais.sort_values('restaurant_id', ascending=False).reset_index(drop=True)
pais.head(1)


,country,restaurant_id
0,India,2177


7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam
reservas?

In [25]:
pais = df[df['has_table_booking'] == 1].groupby('country')['restaurant_id'].count().reset_index()
pais = pais.sort_values('restaurant_id', ascending=False).reset_index(drop=True)
pais.head(1)

,country,restaurant_id
0,India,256


8. Qual o nome do país que possui, na média, a maior quantidade de avaliações
registrada?

In [26]:
pais = df.groupby('country')['votes'].mean().reset_index()
pais = pais.sort_values('votes', ascending=False).reset_index(drop=True)
pais.head(1)


,country,votes
0,Indonesia,1112.825


9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [27]:
pais = df.groupby('country')['aggregate_rating'].mean().reset_index()
pais_max_nota_media = pais.sort_values('aggregate_rating', ascending=False)
pais_max_nota_media.head(1)

,country,aggregate_rating
5,Indonesia,4.60125


In [28]:
pais = df.groupby('country')['aggregate_rating'].mean().sort_values(ascending=False).reset_index()
pais.head(1)

,country,aggregate_rating
0,Indonesia,4.60125


10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [29]:
pais = df.groupby('country')['aggregate_rating'].mean().reset_index()
pais_max_nota_media = pais.sort_values('aggregate_rating', ascending=False)
pais_max_nota_media.head(1)

,country,aggregate_rating
5,Indonesia,4.60125


In [30]:
pais = df.groupby('country')['aggregate_rating'].mean().sort_values(ascending=True).reset_index()
pais.head(1)

,country,aggregate_rating
0,Brazil,3.321667


11. Qual a média de preço de um prato para dois por país?

In [32]:
pais = df.groupby('country')['average_cost_for_two'].mean().sort_values(ascending=False).reset_index()
pais
#px.bar(pais,'country','average_cost_for_two')

,country,average_cost_for_two
0,Indonesia,303000.000000
1,Australia,138959.783333
2,Sri Lanka,2579.375000
3,Philippines,1227.825000
4,India,704.400514
5,South Africa,339.228324
6,Qatar,174.000000
7,United Arab Emirates,153.716667
8,Singapure,141.437500
9,Brazil,138.812500


#### Cidade

1. Qual o nome da cidade que possui mais restaurantes registrados?

2. Qual o nome da cidade que possui mais restaurantes com nota média acima de
4?

3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de
2.5?

4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária
distintas?

6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem
reservas?

7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem
entregas?

8. Qual o nome da cidade que possui a maior quantidade de restaurantes que
aceitam pedidos online?

#### Restaurantes

1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

2. Qual o nome do restaurante com a maior nota média?

3. Qual o nome do restaurante que possui o maior valor de uma prato para duas
pessoas?

4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor
média de avaliação?

5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que
possui a maior média de avaliação?

6. Os restaurantes que aceitam pedido online são também, na média, os
restaurantes que mais possuem avaliações registradas?

7. Os restaurantes que fazem reservas são também, na média, os restaurantes que
possuem o maior valor médio de um prato para duas pessoas?

8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América
possuem um valor médio de prato para duas pessoas maior que as churrascarias
americanas (BBQ)?

#### Tipos de Culinária

1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do
restaurante com a maior média de avaliação?

2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do
restaurante com a menor média de avaliação?

3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do
restaurante com a maior média de avaliação?

4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do
restaurante com a menor média de avaliação?

5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do
restaurante com a maior média de avaliação?

6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do
restaurante com a menor média de avaliação?

7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do
restaurante com a maior média de avaliação?

8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do
restaurante com a menor média de avaliação?

9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do
restaurante com a maior média de avaliação?

10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do
restaurante com a menor média de avaliação?

11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas
pessoas?

12. Qual o tipo de culinária que possui a maior nota média?

13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos
online e fazem entregas?